<a href="https://colab.research.google.com/github/littlerodents/evander/blob/main/AI%E7%90%86%E8%B4%A2%E5%B7%A5%E4%BD%9C%E5%9D%8A%E6%A8%A1%E5%9D%973_%E7%A1%AE%E5%AE%9A%E5%AE%9A%E6%8A%95%E7%AD%96%E7%95%A5_%E6%8A%93%E5%8F%96%E5%9F%BA%E9%87%91%E5%8E%86%E5%8F%B2%E5%87%80%E5%80%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI理财工作坊模块3 确定定投策略-抓取基金历史净值

欢迎参加本次工作坊的第三模块！

在这个模块中，我们将学习如何使用一个简单的 Python 脚本从天天基金网获取指定基金在特定时间范围内的历史净值数据。

我们将按以下步骤进行：
1.  **环境准备**：安装所需的 Python 库。
2.  **定义功能**：载入爬虫的核心函数。
3.  **配置参数**：使用交互式表单设置您想查询的基金代码和日期。
4.  **执行爬虫**：运行脚本获取数据。
5.  **查看与下载**：在 Colab 中直接预览数据，并下载生成的 CSV 文件。

## 步骤 1：环境准备

首先，我们需要安装 `requests` 库，它能帮助我们向网站发送网络请求。`pandas` 库用于更好地展示数据。

In [ ]:
!pip install requests pandas

## 步骤 2：导入库并定义核心功能

接下来，我们导入所有需要的库，并定义两个核心函数：
* `get_fund_data`：负责爬取数据。
* `save_to_csv`：负责将数据保存为 CSV 文件。

这里的代码来源于您提供的 `simple_fund_spider.py` 文件。

In [ ]:
import requests
import re
import csv
import time
from datetime import datetime
import pandas as pd
from google.colab import files

def get_fund_data(fund_code, start_date, end_date):
    """获取基金历史净值数据"""
    print(f"开始获取基金 {fund_code} 的历史净值数据...")
    print(f"日期范围: {start_date} 至 {end_date}")

    url = "http://fund.eastmoney.com/f10/F10DataApi.aspx"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
    }

    all_data = []
    page = 1

    while page <= 20:  # 最多20页
        print(f"正在获取第 {page} 页...")

        params = {
            'type': 'lsjz',
            'code': fund_code,
            'page': page,
            'per': 20,
            'sdate': start_date,
            'edate': end_date
        }

        try:
            response = requests.get(url, params=params, headers=headers, timeout=30)

            # 解析HTML表格
            table_match = re.search(r'<table[^>]*>(.*?)</table>', response.text, re.DOTALL)
            if not table_match:
                break

            rows = re.findall(r'<tr[^>]*>(.*?)</tr>', table_match.group(1), re.DOTALL)

            page_data = []
            for i, row in enumerate(rows[1:]):  # 跳过表头
                cells = re.findall(r'<td[^>]*>(.*?)</td>', row, re.DOTALL)
                if len(cells) >= 4:
                    clean_cells = [re.sub(r'<[^>]*>', '', cell).strip() for cell in cells]
                    if clean_cells[0] and clean_cells[1]:
                        page_data.append({
                            'date': clean_cells[0],
                            'unit_value': clean_cells[1],
                            'accumulated_value': clean_cells[2],
                            'change_percent': clean_cells[3]
                        })

            if not page_data:
                print(f"第 {page} 页无数据，结束")
                break

            all_data.extend(page_data)
            print(f"第 {page} 页获取到 {len(page_data)} 条数据，累计 {len(all_data)} 条")

            if len(page_data) < 20:  # 最后一页
                break

            page += 1
            time.sleep(1)  # 延迟1秒

        except Exception as e:
            print(f"第 {page} 页获取失败: {e}")
            break

    return all_data

def save_to_csv(data, filename):
    """保存数据到CSV文件"""
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=['date', 'unit_value', 'accumulated_value', 'change_percent'])
        writer.writeheader()
        writer.writerows(data)
    print(f"数据已保存到: {filename}")

## 步骤 3：🎛️ 配置参数

请在下方的交互式表单中输入您想要查询的**基金代码**、**开始日期**和**结束日期**。

---

In [ ]:
#@title 请输入您要查询的基金信息 { run: "auto" }
FUND_CODE = "012349" #@param {type:"string"}
START_DATE = "2025-01-01" #@param {type:"date"}
END_DATE = "2025-10-31" #@param {type:"date"}

print(f"设置成功！")
print(f"基金代码 (FUND_CODE): {FUND_CODE}")
print(f"开始日期 (START_DATE): {START_DATE}")
print(f"结束日期 (END_DATE): {END_DATE}")

设置成功！
基金代码 (FUND_CODE): 012349
开始日期 (START_DATE): 2025-01-01
结束日期 (END_DATE): 2025-10-31


---
### 附：常见基金代码参考

（来自 `README_Python.md`）
* `012349`: 易方达蓝筹精选混合
* `000001`: 华夏成长混合
* `110022`: 易方达消费行业股票
* `161725`: 招商中证白酒指数分级

## 步骤 4：🚀 执行爬虫并保存数据

现在，我们将运行一个主函数，它会使用您在**步骤 3** 中设置的参数来调用 `get_fund_data` 和 `save_to_csv`。

In [ ]:
def main():
    # 1. 获取数据
    data = get_fund_data(FUND_CODE, START_DATE, END_DATE)

    if data:
        # 2. 定义文件名
        filename = f"{FUND_CODE}_{START_DATE}_{END_DATE}.csv"

        # 3. 保存到 CSV
        save_to_csv(data, filename)

        # 4. 显示统计信息
        print(f"\n获取完成！共 {len(data)} 条记录")
        print(f"最新数据: {data[0]['date']} - 净值: {data[0]['unit_value']}")
        print(f"最早数据: {data[-1]['date']} - 净值: {data[-1]['unit_value']}")
        return filename
    else:
        print("未获取到数据，请检查基金代码或日期范围。")
        return None

# --- 执行 ---
print("--- 开始执行爬虫 ---")
csv_filename = main()
print("--- 执行完毕 ---")

--- 开始执行爬虫 ---
开始获取基金 012349 的历史净值数据...
日期范围: 2025-01-01 至 2025-10-31
正在获取第 1 页...
第 1 页获取到 20 条数据，累计 20 条
正在获取第 2 页...
第 2 页获取到 20 条数据，累计 40 条
正在获取第 3 页...
第 3 页获取到 20 条数据，累计 60 条
正在获取第 4 页...
第 4 页获取到 20 条数据，累计 80 条
正在获取第 5 页...
第 5 页获取到 20 条数据，累计 100 条
正在获取第 6 页...
第 6 页获取到 20 条数据，累计 120 条
正在获取第 7 页...
第 7 页获取到 20 条数据，累计 140 条
正在获取第 8 页...
第 8 页获取到 20 条数据，累计 160 条
正在获取第 9 页...
第 9 页获取到 20 条数据，累计 180 条
正在获取第 10 页...
第 10 页获取到 20 条数据，累计 200 条
正在获取第 11 页...
第 11 页无数据，结束
数据已保存到: 012349_2025-01-01_2025-10-31.csv

获取完成！共 200 条记录
最新数据: 2025-10-31 - 净值: 0.8352
最早数据: 2025-01-02 - 净值: 0.6323
--- 执行完毕 ---


## 步骤 5：📊 查看与 💾 下载数据

### 5.1 在线预览
为了方便查看，我们使用 `pandas` 库来读取刚刚生成的 CSV 文件，并在下方以表格形式展示数据。

In [ ]:
if csv_filename:
  print(f"--- 正在读取 {csv_filename} ---")
  # 使用 pandas 读取 CSV 文件并展示
  df = pd.read_csv(csv_filename)
  display(df)
else:
  print("没有生成 CSV 文件，无法显示数据。")

--- 正在读取 012349_2025-01-01_2025-10-31.csv ---


,date,unit_value,accumulated_value,change_percent
0,2025-10-31,0.8352,0.8352,-2.22%
1,2025-10-30,0.8542,0.8542,-0.64%
2,2025-10-29,0.8597,0.8597,-0.03%
3,2025-10-28,0.8600,0.8600,-1.22%
4,2025-10-27,0.8706,0.8706,1.73%
...,...,...,...,...
195,2025-01-08,0.6252,0.6252,-1.06%
196,2025-01-07,0.6319,0.6319,-0.83%
197,2025-01-06,0.6372,0.6372,-0.19%
198,2025-01-03,0.6384,0.6384,0.96%


### 5.2 下载文件

运行下方单元格，您的浏览器将自动开始下载已生成的 `.csv` 文件。

In [ ]:
if csv_filename:
  print(f"点击以下链接下载文件: {csv_filename}")
  files.download(csv_filename)
else:
  print("没有文件可供下载。")

点击以下链接下载文件: 012349_2025-01-01_2025-10-31.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---
🎉 工作坊内容结束！